Let's install a bunch of stuff. You'll need a virtual environment activated, or be running as root, otherwise your system will yell at you.

In [ ]:
!pip install requests networkx matplotlib;

Cool, now let's import a bunch of stuff, and tell matplotlib that we want to see the graphs.

In [ ]:
import os
import json


import requests
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

Let's try and find the community following my Github.
We'll need the `requests` library.

In [ ]:
github_graphql_url = 'https://api.github.com/graphql'

# I'm using envirnomental variables so I can distribute this without leaking my token
# a normal string will do
oauth_token = os.getenv('GITHUB_OAUTH_TOKEN')

# format strings in python 3.6+ are pretty neat
headers={'Authorization': f'bearer {oauth_token}'}

In [ ]:
query = '''
viewer {
  login
  followers (first: 100)
  {
    pageInfo {
      hasNextPage
      endCursor
    }
    nodes {
      login
      following (first: 100) {
        pageInfo {
          hasNextPage
          endCursor
        }
        edges {
          node {
            login
          }
        }
      }
    }
  }  
}
'''

In [ ]:
def get_data(query):
    r = requests.post(url, data=json.dumps({'query': query}), headers=headers)
    print(r.json())
    return r.json()['data']

# TODO: add cursor logic
def get_follower_and_login_data(cursor=None):
    data = get_data(query)
    user_login_name = data['viewer']['login']
    my_follower_data = data_request_dict['viewer']['followers']
    return my_follower_data, user_login_name

In [ ]:
my_follower_data, user_login_name = get_follower_and_login_data()

Define a few helper functions to deal with our deeply nested dictionaries

In [ ]:
def get_followers_and_next_page(my_follower_data: dict) -> (list, bool):
    # who is following me?
    my_followers = my_follower_data['nodes']
    my_followers = [x['login'] for x in my_followers]
    # do I have more than one page of followers?
    has_next_page = my_follower_data['pageInfo']['hasNextPage']
    # what is the cursor string that for use in the next query?
    cursor = my_follower_data['pageInfo']['endCursor']
    
    return my_followers, has_next_page, cursor

def get_first_layer_followers_and_next_page(a_follower: dict) -> (list, bool):
    # Who is this user following?
    following = a_follower['following']['edges']
    following = [x['node']['login'] for x in following]
    # Is there a next page?
    has_next_page = following['pageInfo']['hasNextPage']
    # what is the cursor string that for use in the next query?
    cursor = my_follower_data['pageInfo']['endCursor']
    
    return following, has_next_page, cursor

In [ ]:
# setup our graph stuff
graph = nx.Graph()
graph.add_node(user_login_name)

In [ ]:
has_next_page = True
on_last_page = False
followers_with_many_following = {}

while has_next_page or not on_last_page:
    my_followers, has_next_page, cursor = get_followers_and_next_page(my_follower_data)

    # Add all of my followers to the graph, and capture the connection
    for follower in my_followers:
        graph.add_node(follower)
        graph.add_edge(follower, user_login_name)

    for follower in my_followers:
        first_layer, first_layer_has_next_page, first_layer_cursor = get_first_layer_followers_and_next_page(follower)
        for login in first_layer:
            graph.add_node(login)
            graph.add_edge(follower, login)
        if has_next_page:
            followers_with_many_following[login] = first_layer_cursor
            
    # Loop maintainence
    if has_next_page == False:
        on_last_page = True
    else:
        my_follower_data, _ = get_follower_and_login_data(cursor)
        
while followers_with_many_following:
    keys = tuple(followers_with_many_following.keys())
    for key in keys:
        cursor = followers_with_many_following[key]
        data = get_data(user_query.format(login=key, cursor=cursor))
    followers_with_many_following.pop(key)

In [ ]:
user_query =
'''
repositoryOwner(login: "{login"){
  ... on User {
    following(first: 100 after: {cursor}) {
      totalCount
      pageInfo {
        endCursor
        hasNextPage
      }
      edges {
        node {
          login
        }
      }
    }
  }
}
'''

In [ ]:
nx.draw_spectral(graph)
plt.show() 